In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from trajectory_visualization import plot_trajectory, sidebyside, shift_trajectory, overlay, shift_trajectory3D
from trajectory_visualization import plot_trajectories3D, plot_3Doverlay, plot_MSDorDeff, plot_MeanMSDorDeff, randtraj, multrandtraj
from trajectory_visualization import randtraj2, plot_Mean2DMSDsorDeff, plot_MSDorDeffLR, LRfor3D2D, fillin, prettify
from methods_of_Dcalc import Dpointder, Dlinalg, Dlinalgw
from Diffusion_Functions import frame_adjust, MSD_Deff_Calcs, prettify2, fillin2

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import scipy.optimize as opt
import scipy.stats as stat
from mpl_toolkits.mplot3d import Axes3D
from operator import itemgetter
import random
import numpy as np
import numpy.ma as ma
import numpy.linalg as la
from mpl_toolkits.mplot3d import Axes3D

pi = np.pi
sin = np.sin
cos = np.cos

#I found an error that was causing huge problems with the arithmetic mean. There was one particle with a huge jump
#That caused the MSDs to be really high (20 um jump? Crazy!).  We need to create some functionality to eliminate those 
#Obviously fake particles.  I will create a short code to filter particles with huge MSDS. 

In [2]:
#Currently, my code only has capability to do frame-by-frame analysis on 1 video.  Perhaps future functionality will
#be able to expand beyond this.  

name = 'CAM_40x_2D_200ms'

logplot = '{}_logplot'.format(name)
Mplot = '{}_Mplot'.format(name)
Dplot = '{}_Dplot'.format(name)
Hplot = '{}_Hplot'.format(name)
Hlogplot = '{}_Hlogplot'.format(name)
Cplot = '{}_Cplot'.format(name)
Tplot = '{}_Tplot'.format(name)

conversion = dict() #First element is the umppx, second is fps

conversion[1] = (0.16, 5.02, 1)
#conversion[2] = (0.16, 5.02, 1)
#conversion[3] = (0.16, 5.02, 1)
# conversion[4] = (0.30, 5.31, 1)
# conversion[5] = (0.30, 5.33, 1)
# conversion[6] = (0.30, 5.33, 1)
# conversion[7] = (0.30, 5.35, 1)

trajectory = dict()
cut = 5
great = 50

#Particles that you want manually filtered.
filtered = False
tofilt = np.array([4,  36,  82,  92, 126, 148, 151, 161, 162, 186, 209, 223, 234,
       249, 266, 286, 298, 313, 316, 317, 334, 337, 340, 378, 431, 523,
    32,  83, 114, 156, 176, 194, 214, 253, 260, 315, 320, 322, 332,
       343, 371, 373, 377, 494])

trajectory[1] = np.genfromtxt('../../sample_data/12-07-06-75p-glycerol-exp/CAM_40x_2D_200ms_2_T.csv',
            delimiter =",")
#trajectory[2] = np.genfromtxt('./confocal_1000ms_40x_2.csv',
#            delimiter =",")
#trajectory[3] = np.genfromtxt('./confocal_1000ms_40x_3.csv',
#            delimiter =",")
# trajectory[4] = np.genfromtxt('./CON_40x_2D_130ms_4_T.csv',
#             delimiter =",")
# trajectory[5] = np.genfromtxt('./CON_40x_2D_130ms_5_T.csv',
#             delimiter =",")
# trajectory[6] = np.genfromtxt('./CON_40x_2D_130ms_6_T.csv',
#             delimiter =",")
# trajectory[7] = np.genfromtxt('./CON_40x_2D_130ms_7_T.csv',
#             delimiter =",")

totvids = 1

for num in range(1, totvids + 1):
    
    #Remove titles from columns
    #trajectory[num]=np.delete(trajectory[num], 0,0)
    #Remove the number column from dataset
    trajectory[num]=np.delete(trajectory[num],0,1)

#Collect trajectories of separate videos into one array
#Comment this section out to make useable for a single trajectory
parts = dict()
tots = dict()
newtots = dict()
newtots[0] = 0
tlen = dict()
tlength = dict()
tlength[0] = 0

for num in range(1, totvids + 1):
    tots[num] = trajectory[num][-1, 0].astype(np.int64)
    parts[num] = tots[num]
    counter = 1
    newtots[num] = newtots[num-1] + tots[num]
    
    tlen[num] = trajectory[num].shape[0]
    tlength[num] = tlength[num-1] + tlen[num]

placeholder = np.zeros((tlength[totvids], 11))

#     while counter < parts[num]:
#         counter = counter + 1
#         placeholder = np.append(lehi[num], final[num][counter], axis=0)

for num in range(1, totvids + 1):
    placeholder[tlength[num-1]:tlength[num], :] = trajectory[num]
    placeholder[tlength[num-1]:tlength[num], 0] = placeholder[tlength[num-1]:tlength[num], 0] + newtots[num-1]

# trajectory[1] = placeholder

# trajectory[2][:,0] = trajectory[2][:,0] + tots[1]
# placeholder = np.append(placeholder, trajectory[2], axis=0)
# trajectory[3][:,0] = trajectory[3][:,0] + tots[2]
# placeholder = np.append(placeholder, trajectory[3], axis=0)

In [3]:
placeholder[-1, :]

array([  7.44000000e+02,   4.50000000e+01,   9.59650000e+01,
         3.75100000e+01,   0.00000000e+00,   5.94000000e-01,
         1.04200000e+00,   1.50900000e+00,   2.38600000e+00,
         4.06000000e+00,   5.60151000e+02])

In [4]:
trajectory[3].shape

(5385, 11)

In [5]:
newtots

{0: 0, 1: 231, 2: 505, 3: 744}

In [6]:
6206+6075+5385

17666

In [7]:
#Theoretical diffusion calculation using Stokes-Einstein

k = 1.380649 * 10**-23
TC = 37
TK = TC + 273
mu1 = 1.002*10**-3; mu2 = 0.798*10**-3; TC1 = 20; TC2 = 30

mu = (mu2 - mu1)*(TC - TC1)/(TC2 - TC1) + mu1
radius = (128/2) * 10**-9 #m
Deff1 = k*TK/(6*np.pi*mu*radius) #m2/s
Deff2 = Deff1 * 10**12 #um2/s
Deff2

5.414894389572004

In [8]:
def fillin2(data):
    """
    Fills in blanks of arrays without shifting frames by the starting frame.  Compare to fillin.
    """
    shap = int(max(data[:, 1])) + 1
    shape1 = int(min(data[:, 1]))
    newshap = shap - shape1
    filledin = np.zeros((newshap, 5))
    filledin[0, :] = data[0, :]

    count = 0
    new = 0
    other = 0
    tot = 0

    for num in range(1, newshap):
        if data[num-new, 1]-data[num-new-1, 1] == 1:
            count = count + 1
            filledin[num, :] = data[num-new, :]
        elif data[num - new, 1]-data[num - new - 1, 1] > 1:
            new = new + 1
            iba = int(data[num - new+1, 1]-data[num - new, 1])
            togoin = data[num - new]
            togoin[1] = togoin[1] + 1
            filledin[num, :] = togoin
            # dataset[2] = np.insert(dataset[2], [num + new - 2], togoin, axis=0)

        else:
            other = other + 1
        tot = count + new + other

    return filledin

In [9]:
rawframes = placeholder[:, 1]
#frames = np.linspace()
frames = np.linspace(min(rawframes),max(rawframes), max(rawframes)+1).astype(np.int64)
time = frames / conversion[1][1]

x = dict()
y = dict()
z = dict()

xs = dict()
ys = dict()
zs = dict()

M1x = dict()
M1y = dict()
M1z = dict()
M2xy = dict()
M2xz = dict()
M2yz = dict()
M3 = dict()

#Shifted dictionaries
SM1x = dict()
SM1y = dict()
SM1z = dict()
SM2xy = dict()
SM2xz = dict()
SM2yz = dict()
SM3 = dict()

SD1x = dict()
SD1y = dict()
SD1z = dict()
SD2xy = dict()
SD2xz = dict()
SD2yz = dict()
SD3 = dict()

In [10]:
time.shape

(46,)

In [11]:
dataset = dict()

rawdataset = np.zeros(placeholder.shape)
particles = placeholder[:, 0]
total = int(max(particles))
total1 = total + 1
rawdataset = placeholder[:, :]

#Perform conversion on raw dataset
# rawdataset[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
# rawdataset[:, 4] = conversion[1][2] * rawdataset[:, 4]

fixed = np.zeros(placeholder.shape)
fixed[:, 0:2] = rawdataset[:, 0:2]
fixed[:, 2:4] = conversion[1][0] * rawdataset[:, 2:4]
fixed[:, 4] = conversion[1][2] * rawdataset[:, 4]

# Creates an array for each trajectory containing all xyz data
for num in range(1, total1):

    hold = np.where(particles == num)
    itindex = hold[0]
    min1 = min(itindex)
    max1 = max(itindex)
    dataset[num] = (fixed[min1:max1+1, 0:5])

In [12]:
dataset[540]

array([[ 540.     ,    0.     ,  294.90796,   26.8646 ,    0.     ],
       [ 540.     ,    1.     ,  295.01832,   26.94892,    0.     ],
       [ 540.     ,    2.     ,  294.89556,   26.89064,    0.     ],
       [ 540.     ,    3.     ,  294.89184,   26.93032,    0.     ],
       [ 540.     ,    4.     ,  294.81372,   26.94148,    0.     ],
       [ 540.     ,    5.     ,  294.86456,   26.877  ,    0.     ],
       [ 540.     ,    6.     ,  294.92284,   26.91792,    0.     ],
       [ 540.     ,    7.     ,  294.91788,   26.7778 ,    0.     ],
       [ 540.     ,    8.     ,  294.8286 ,   26.91048,    0.     ],
       [ 540.     ,    9.     ,  295.01832,   26.86584,    0.     ],
       [ 540.     ,   10.     ,  294.90052,   26.92784,    0.     ],
       [ 540.     ,   11.     ,  295.00344,   26.9948 ,    0.     ],
       [ 540.     ,   12.     ,  294.8968 ,   26.75796,    0.     ],
       [ 540.     ,   13.     ,  295.027  ,   26.84352,    0.     ],
       [ 540.     ,   14.     ,  2

In [13]:
#I is a set of all nonzero elements in trajectories.
I = dict()
for num in range(1, total1):
    #Construct x, y, z
    dataset[num] = fillin2(dataset[num])
    x[num] = np.zeros(frames.shape[0])
    x[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 2]
    y[num] = np.zeros(frames.shape[0])
    y[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 3]
    z[num] = np.zeros(frames.shape[0])
    z[num][int(dataset[num][0,1]):int(dataset[num][-1,1])+1] = dataset[num][:, 4]
    
    xs[num] = np.zeros(frames.shape[0])
    xs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 2]
    ys[num] = np.zeros(frames.shape[0])
    ys[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 3]
    zs[num] = np.zeros(frames.shape[0])
    zs[num][0:int(dataset[num][-1,1])+1-int(dataset[num][0,1])] = dataset[num][:, 4]

In [14]:
np.count_nonzero(x[300])

46

In [15]:
#I wrote a short bit of code to filter out trajectories that are too short to be included.  This also reduces computing
#time by cutting down on MSDs that need to be calculated.  

cutoff = cut
x1 = dict()
y1 = dict()
z1 = dict()

xs1 = dict()
ys1 = dict()
zs1 = dict()

fifties = 0
nones = 0

for num in range(1, total1):
    if np.count_nonzero(x[num]) < cutoff:
        nones = nones + 1
    else:
        fifties = fifties + 1
        x1[num - nones] = x[num]
        y1[num - nones] = y[num]
        z1[num - nones] = z[num]
        
        xs1[num - nones] = xs[num]
        ys1[num - nones] = ys[num]
        zs1[num - nones] = zs[num]
        # I must also redefine the particle numbers to reflect the new set.
        #teancum[num - nones][:, 0] = fifties

x = x1
y = y1
z = z1

xs = xs1
ys = ys1
zs = zs1

In [16]:
x[219]

array([ 284.24768,  284.425  ,  284.3258 ,  284.25884,  284.43864,
        284.44856,  284.3134 ,  284.32704,  284.32084,  284.32456,
        284.42252,  284.30472,  284.27496,  284.4932 ,  284.36796,
        284.239  ,  284.2204 ,  284.44608,  284.30472,  284.19684,
        284.2886 ,  284.3878 ,  284.1894 ,  284.15096,  284.394  ,
        284.24272,  284.28984,  284.1584 ,  284.19808,  284.20676,
        284.27124,  284.26628,  284.14848,  284.22908,  284.30596,
        284.4126 ,  284.45848,  284.4002 ,  284.10632,  284.25884,
        284.21668,  284.22908,  284.19312,  284.29356,  284.31712,
        284.27124])

In [17]:
#fifties is the new total1 after filtering out short trajectories
total1 = fifties + 1
fifties

542

In [18]:
xymask = dict()

for num in range(1, total1):

    #Construct MSD and Deff dictionaries
    M1x[num] = np.zeros(frames.shape[0])
    M1x[num][0:frames.shape[0]] = 0
    M1y[num] = np.zeros(frames.shape[0])
    M1y[num][0:frames.shape[0]] = 0
    M1z[num] = np.zeros(frames.shape[0])
    M1z[num][0:frames.shape[0]] = 0
    M2xy[num] = np.zeros(frames.shape[0])
    M2xy[num][0:frames.shape[0]] = 0
    M2xz[num] = np.zeros(frames.shape[0])
    M2xz[num][0:frames.shape[0]] = 0
    M2yz[num] = np.zeros(frames.shape[0])
    M2yz[num][0:frames.shape[0]] = 0
    M3[num] = np.zeros(frames.shape[0])
    M3[num][0:frames.shape[0]] = 0
    
    SM1x[num] = np.zeros(frames.shape[0])
    SM1y[num] = np.zeros(frames.shape[0])
    SM1z[num] = np.zeros(frames.shape[0])
    SM2xy[num] = np.zeros(frames.shape[0])
    SM2xz[num] = np.zeros(frames.shape[0])
    SM2yz[num] = np.zeros(frames.shape[0])
    SM3[num] = np.zeros(frames.shape[0])
    
    SD1x[num] = np.zeros(frames.shape[0])
    SD1y[num] = np.zeros(frames.shape[0])
    SD1z[num] = np.zeros(frames.shape[0])
    SD2xy[num] = np.zeros(frames.shape[0])
    SD2xz[num] = np.zeros(frames.shape[0])
    SD2yz[num] = np.zeros(frames.shape[0])
    SD3[num] = np.zeros(frames.shape[0])
    
    #Fill in MSD and Deff dictionaries
    I[num] = np.nonzero(x[num])[0]
    first = I[num][0]
    last = I[num][-1] + 1
    startx = x[num][first]
    starty = y[num][first]
    startz = z[num][first]
    
    for num1 in range(first, last):
        M1x[num][num1] = (x[num][num1] - startx)**2
        M1y[num][num1] = (y[num][num1] - starty)**2
        M1z[num][num1] = (z[num][num1] - startz)**2
        M2xy[num][num1] = (x[num][num1] - startx)**2 + (y[num][num1] - starty)**2
        M2xz[num][num1] = (x[num][num1] - startx)**2 + (z[num][num1] - startz)**2
        M2yz[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2
        M3[num][num1] = (z[num][num1] - startz)**2 + (y[num][num1] - starty)**2 + (x[num][num1] - startx)**2
        
#         M1x[num][first] = 'NaN'
#         M1y[num][first] = 'NaN'
#         M1z[num][first] = 'NaN'
#         M2xy[num][first] = 'NaN'
#         M2xz[num][first] = 'NaN'
#         M2yz[num][first] = 'NaN'
#         M3[num][first] = 'NaN'
    M1x[num] = ma.masked_equal(M1x[num], 0)
    M1y[num] = ma.masked_equal(M1y[num], 0)
    M1z[num] = ma.masked_equal(M1z[num], 0)
    M2xy[num] = ma.masked_equal(M2xy[num], 0)
    M2xz[num] = ma.masked_equal(M2xz[num], 0)
    M2yz[num] = ma.masked_equal(M2yz[num], 0)
    M3[num] = ma.masked_equal(M3[num], 0)
    
    M1x[num] = ma.masked_greater(M1x[num], great)
    M1y[num] = ma.masked_greater(M1y[num], great)
    M1z[num] = ma.masked_greater(M1z[num], great)
    M2xy[num] = ma.masked_greater(M2xy[num], great)
    M2xz[num] = ma.masked_greater(M2xz[num], great)
    M2yz[num] = ma.masked_greater(M2yz[num], great)
    M3[num] = ma.masked_greater(M3[num], great)
    
    #Make shifted MSD arrays for geometric and arithmetic averages.
    
    if M2xy[num][~M2xy[num].mask].shape[0] != 0:
        fir = np.nonzero(M2xy[num])[0][0] - 1
        firl = M1x[num][fir:].shape[0]
    else:
        fir = 0
        firl = 46
    
    SM1x[num][0:firl] = M1x[num][fir:]
    SM1y[num][0:firl] = M1y[num][fir:]
    SM1z[num][0:firl] = M1z[num][fir:]
    SM2xy[num][0:firl] = M2xy[num][fir:]
    SM2xz[num][0:firl] = M2xz[num][fir:]
    SM2yz[num][0:firl] = M2yz[num][fir:]
    SM3[num][0:firl] = M3[num][fir:]

    SM1x[num] = ma.masked_greater(SM1x[num], great)
    SM1y[num] = ma.masked_greater(SM1y[num], great)
    SM1z[num] = ma.masked_greater(SM1z[num], great)
    SM2xy[num] = ma.masked_greater(SM2xy[num], great)
    SM2xz[num] = ma.masked_greater(SM2xz[num], great)
    SM2yz[num] = ma.masked_greater(SM2yz[num], great)
    SM3[num] = ma.masked_greater(SM3[num], great)

    xymask[num] = SM2xy[num].recordmask
    xs[num] = ma.array(xs[num], mask = xymask[num])
    ys[num] = ma.array(ys[num], mask = xymask[num])
    zs[num] = ma.array(zs[num], mask = xymask[num])
    
    xs[num] = ma.masked_equal(xs[num], 0)
    ys[num] = ma.masked_equal(ys[num], 0)
    zs[num] = ma.masked_equal(zs[num], 0)
    
    
    SD1x[num][1:] = SM1x[num][1:]/(2*time[1:])
    SD1y[num][1:] = SM1y[num][1:]/(2*time[1:])
    SD1z[num][1:] = SM1z[num][1:]/(2*time[1:])
    SD2xy[num][1:] = SM2xy[num][1:]/(4*time[1:])
    SD2xz[num][1:] = SM2xz[num][1:]/(4*time[1:])
    SD2yz[num][1:] = SM2yz[num][1:]/(4*time[1:])
    SD3[num][1:] = SM3[num][1:]/(6*time[1:])

    SM1x[num] = ma.masked_equal(SM1x[num], 0)
    SM1y[num] = ma.masked_equal(SM1y[num], 0)
    SM1z[num] = ma.masked_equal(SM1z[num], 0)
    SM2xy[num] = ma.masked_equal(SM2xy[num], 0)
    SM2xz[num] = ma.masked_equal(SM2xz[num], 0)
    SM2yz[num] = ma.masked_equal(SM2yz[num], 0)
    SM3[num] = ma.masked_equal(SM3[num], 0)

    SD1x[num] = ma.masked_equal(SD1x[num], 0)
    SD1y[num] = ma.masked_equal(SD1y[num], 0)
    SD1z[num] = ma.masked_equal(SD1z[num], 0)
    SD2xy[num] = ma.masked_equal(SD2xy[num], 0)
    SD2xz[num] = ma.masked_equal(SD2xz[num], 0)
    SD2yz[num] = ma.masked_equal(SD2yz[num], 0)
    SD3[num] = ma.masked_equal(SD3[num], 0)

In [19]:
#Manually mask particle trajectories here that seem out of whack.

if filtered == True:
    for num in range(0, tofilt.shape[0]):
        M1x[tofilt[num]] = ma.masked_greater(M1x[tofilt[num]], 0)
        M1y[tofilt[num]] = ma.masked_greater(M1y[tofilt[num]], 0)
        M1z[tofilt[num]] = ma.masked_greater(M1z[tofilt[num]], 0)
        M2xy[tofilt[num]] = ma.masked_greater(M2xy[tofilt[num]], 0)
        M2xz[tofilt[num]] = ma.masked_greater(M2xz[tofilt[num]], 0)
        M2yz[tofilt[num]] = ma.masked_greater(M2yz[tofilt[num]], 0)
        M3[tofilt[num]] = ma.masked_greater(M3[tofilt[num]], 0)
    
        SM1x[tofilt[num]] = ma.masked_greater(SM1x[tofilt[num]], 0)
        SM1y[tofilt[num]] = ma.masked_greater(SM1y[tofilt[num]], 0)
        SM1z[tofilt[num]] = ma.masked_greater(SM1z[tofilt[num]], 0)
        SM2xy[tofilt[num]] = ma.masked_greater(SM2xy[tofilt[num]], 0)
        SM2xz[tofilt[num]] = ma.masked_greater(SM2xz[tofilt[num]], 0)
        SM2yz[tofilt[num]] = ma.masked_greater(SM2yz[tofilt[num]], 0)
        SM3[tofilt[num]] = ma.masked_greater(SM3[tofilt[num]], 0)
    
        SD1x[tofilt[num]] = ma.masked_greater(SD1x[tofilt[num]], 0)
        SD1y[tofilt[num]] = ma.masked_greater(SD1y[tofilt[num]], 0)
        SD1z[tofilt[num]] = ma.masked_greater(SD1z[tofilt[num]], 0)
        SD2xy[tofilt[num]] = ma.masked_greater(SD2xy[tofilt[num]], 0)
        SD2xz[tofilt[num]] = ma.masked_greater(SD2xz[tofilt[num]], 0)
        SD2yz[tofilt[num]] = ma.masked_greater(SD2yz[tofilt[num]], 0)
        SD3[tofilt[num]] = ma.masked_greater(SD3[tofilt[num]], 0)
        
        xs[tofilt[num]] = ma.masked_greater(xs[tofilt[num]], 0)
        ys[tofilt[num]] = ma.masked_greater(ys[tofilt[num]], 0)
        zs[tofilt[num]] = ma.masked_greater(zs[tofilt[num]], 0)

In [20]:
#np.count_nonzero(M1x[495])
shift = next((i for i, x in enumerate(M1x[50]) if x), None)
sM1x = dict()
sM1x[50] = np.zeros(frames.shape[0] - shift)
for num in range(0, M1x[50].shape[0] - shift):
    sM1x[50][num] = M1x[50][num + shift]
sM1x

{50: array([  4.31911840e-03,   5.91053440e-03,   2.63867536e-02,
          3.14423824e-02,   9.11643040e-03,   1.88356000e-03,
          5.17248640e-03,   7.75104160e-03,   3.01369600e-04,
          1.77746560e-03,   2.71232640e-03,   3.93625600e-04,
          1.20547840e-03,   6.90228640e-03,   1.63123984e-02,
          1.29312160e-03,   2.47999504e-02,   6.49636000e-03,
          1.86049600e-04,   3.39655840e-03,   2.22029440e-03,
          1.38768400e-02,   7.53424000e-03,   1.77746560e-03,
          1.20547840e-03,   1.63123984e-02,   1.63123984e-02,
          3.45960000e-04,   1.00881936e-02,   3.69177760e-03,
          7.53424000e-03,   1.99272960e-03,   6.15040000e-06,
          6.15040000e-04,   1.53760000e-04,   2.59854400e-04,
          7.53424000e-05,   2.58470560e-03,   8.64900000e-03,
          1.20547840e-03,   1.57450240e-03,   1.77746560e-03,
          2.59854400e-04,   1.16380944e-02,   6.15040000e-06])}

In [21]:
#Each entry in yes is an array of particles that are in the given frame.  The frame is given by the index of yes.
yes = dict()
yessize = dict()
for num1 in range(1, frames.shape[0]+1):
    count = 0
    yes[num1] = np.zeros(total1).astype(np.int64)
    for num in range(1, total1):
        check = num1 in I[num]
        if check == True:
            yes[num1][count] = num
            count = count + 1
    iindex = np.where(yes[num1]==max(yes[num1]))[0][0]
    yes[num1] = yes[num1][0:iindex]

In [22]:
#And the FBF analyis
framed = dict()
fM1y = dict()
fM1z = dict()
fM2xy = dict()
fM2xz = dict()
fM2yz = dict()
fM3 = dict()

#Due to some issues with the first frame being a NaN, my for loop is cut one short.  Get funky results otherwise.
for num2 in range(1, frames.shape[0]):
    framed[num2] = np.zeros(frames.shape[0])
    fM1y[num2] = np.zeros(frames.shape[0])
    fM1z[num2] = np.zeros(frames.shape[0])
    fM2xy[num2] = np.zeros(frames.shape[0])
    fM2xz[num2] = np.zeros(frames.shape[0])
    fM2yz[num2] = np.zeros(frames.shape[0])
    fM3[num2] = np.zeros(frames.shape[0])
    
    for num1 in range(0, frames.shape[0]):
        frame = np.zeros(yes[num2].shape[0])
        fmM1y = np.zeros(yes[num2].shape[0])
        fmM1z = np.zeros(yes[num2].shape[0])
        fmM2xy = np.zeros(yes[num2].shape[0])
        fmM2xz = np.zeros(yes[num2].shape[0])
        fmM2yz = np.zeros(yes[num2].shape[0])
        fmM3 = np.zeros(yes[num2].shape[0])
        
        for num in range(0, yes[num2].shape[0]):
            frame[num] = M1x[yes[num2][num]][num1]
            fmM1y[num] = M1y[yes[num2][num]][num1]
            fmM1z[num] = M1z[yes[num2][num]][num1]
            fmM2xy[num] = M2xy[yes[num2][num]][num1]
            fmM2xz[num] = M2xz[yes[num2][num]][num1]
            fmM2yz[num] = M2yz[yes[num2][num]][num1]
            fmM3[num] = M3[yes[num2][num]][num1]
        frame = ma.masked_invalid(frame)
        fmM1y = ma.masked_invalid(fmM1y)
        fmM1z = ma.masked_invalid(fmM1z)
        fmM2xy = ma.masked_invalid(fmM2xy)
        fmM2xz = ma.masked_invalid(fmM2xz)
        fmM2yz = ma.masked_invalid(fmM2yz)
        fmM3 = ma.masked_invalid(fmM3)
        
        framed[num2][num1] = stat.gmean(frame)
        fM1y[num2][num1] = stat.gmean(fmM1y)
        fM1z[num2][num1] = stat.gmean(fmM1z)
        fM2xy[num2][num1] = stat.gmean(fmM2xy)
        fM2xz[num2][num1] = stat.gmean(fmM2xz)
        fM2yz[num2][num1] = stat.gmean(fmM2yz)
        fM3[num2][num1] = stat.gmean(fmM3)

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [23]:
#Old way of calculating the FBF averages.

# FM1x = np.zeros(frames.shape[0])
# FM1y = np.zeros(frames.shape[0])
# FM1z = np.zeros(frames.shape[0])
# FM2xy = np.zeros(frames.shape[0])
# FM2xz = np.zeros(frames.shape[0])
# FM2yz = np.zeros(frames.shape[0])
# FM3 = np.zeros(frames.shape[0])

# for num in range(1, frames.shape[0]):
#     FM1x = FM1x + framed[num]
#     FM1y = FM1y + fM1y[num]
#     FM1z = FM1z + fM1z[num]
#     FM2xy = FM2xy + fM2xy[num]
#     FM2xz = FM2xz + fM2xz[num]
#     FM2yz = FM2yz + fM2yz[num]
#     FM3 = FM3 + fM3[num]
    
# FM1x = FM1x/frames.shape[0]
# FM1y = FM1y/frames.shape[0]
# FM1z = FM1z/frames.shape[0]
# FM2xy = FM2xy/frames.shape[0]
# FM2xz = FM2xz/frames.shape[0]
# FM2yz = FM2yz/frames.shape[0]
# FM3 = FM3/frames.shape[0]



In [24]:
FM1x = np.zeros(frames.shape[0])
FM1y = np.zeros(frames.shape[0])
FM1z = np.zeros(frames.shape[0])
FM2xy = np.zeros(frames.shape[0])
FM2xz = np.zeros(frames.shape[0])
FM2yz = np.zeros(frames.shape[0])
FM3 = np.zeros(frames.shape[0])

st_FM1x = np.zeros(frames.shape[0])
st_FM1y = np.zeros(frames.shape[0])
st_FM1z = np.zeros(frames.shape[0])
st_FM2xy = np.zeros(frames.shape[0])
st_FM2xz = np.zeros(frames.shape[0])
st_FM2yz = np.zeros(frames.shape[0])
st_FM3 = np.zeros(frames.shape[0])

swframed = dict()
swfM1y = dict()
swfM1z = dict()
swfM2xy = dict()
swfM2xz = dict()
swfM2yz = dict()
swfM3 = dict()

for num in range(1, frames.shape[0]+1):
    swframed[num] = np.zeros(frames.shape[0]-1)
    swfM1y[num] = np.zeros(frames.shape[0]-1)
    swfM1z[num] = np.zeros(frames.shape[0]-1)
    swfM2xy[num] = np.zeros(frames.shape[0]-1)
    swfM2xz[num] = np.zeros(frames.shape[0]-1)
    swfM2yz[num] = np.zeros(frames.shape[0]-1)
    swfM3[num] = np.zeros(frames.shape[0]-1)
    
    for num1 in range(1, frames.shape[0]):
        swframed[num][num1-1] = framed[num1][num-1]
        swfM1y[num][num1-1] = fM1y[num1][num-1]
        swfM1z[num][num1-1] = fM1z[num1][num-1]
        swfM2xy[num][num1-1] = fM2xy[num1][num-1]
        swfM2xz[num][num1-1] = fM2xz[num1][num-1]
        swfM2yz[num][num1-1] = fM2yz[num1][num-1]
        swfM3[num][num1-1] = fM3[num1][num-1]

for num in range(1, frames.shape[0]+1):
    FM1x[num-1] = np.mean(swframed[num])
    FM1y[num-1] = np.mean(swfM1y[num])
    FM1z[num-1] = np.mean(swfM1z[num])
    FM2xy[num-1] = np.mean(swfM2xy[num])
    FM2xz[num-1] = np.mean(swfM2xz[num])
    FM2yz[num-1] = np.mean(swfM2yz[num])
    FM3[num-1] = np.mean(swfM3[num])

    st_FM1x[num-1] = np.std(swframed[num])
    st_FM1y[num-1] = np.std(swfM1y[num])
    st_FM1z[num-1] = np.std(swfM1z[num])
    st_FM2xy[num-1] = np.std(swfM2xy[num])
    st_FM2xz[num-1] = np.std(swfM2xz[num])
    st_FM2yz[num-1] = np.std(swfM2yz[num])
    st_FM3[num-1] = np.std(swfM3[num])

In [25]:
ty = dict()

for num in range(1, frames.shape[0]):
    ty[num] = np.zeros(frames.shape[0]-1)
    
    for num1 in range(1, frames.shape[0]):
        ty[num][num1-1] = framed[num1][num-1]

ty[45].shape

(45,)

In [26]:
swframed[46]

array([ 0.00480966,  0.00480966,  0.00480966,  0.00480966,  0.00475899,
        0.00475899,  0.00475899,  0.00475899,  0.00475899,  0.00475899,
        0.00475899,  0.00475899,  0.00473374,  0.00476918,  0.00476918,
        0.00476918,  0.00476918,  0.00476918,  0.00477731,  0.00477731,
        0.00477731,  0.00479031,  0.00480719,  0.00480719,  0.00480719,
        0.00480719,  0.00488113,  0.00493024,  0.00494329,  0.00495766,
        0.00499457,  0.00503251,  0.00503251,  0.00499205,  0.00494807,
        0.00499139,  0.00502388,  0.00498949,  0.00498949,  0.00502337,
        0.00508478,  0.00508478,  0.00508478,  0.00508478,  0.00508478])

In [27]:
FM1x

array([        nan,  0.00240835,  0.00139966,  0.00189864,  0.0020374 ,
        0.00214677,  0.00200484,  0.002061  ,  0.00167364,  0.00192637,
        0.00228802,  0.00214348,  0.00218059,  0.00233437,  0.00244532,
        0.00229995,  0.00263061,  0.00314033,  0.00246232,  0.00277585,
        0.00273753,  0.00280447,  0.00326733,  0.00324396,  0.00378958,
        0.00345684,  0.00354039,  0.0035148 ,  0.00323551,  0.00341907,
        0.00435647,  0.00409122,  0.00316787,  0.00321058,  0.00378844,
        0.00412293,  0.00300384,  0.00378123,  0.00418654,  0.00465427,
        0.00439011,  0.00488274,  0.00399087,  0.00449355,  0.00370223,
        0.00487544])

In [28]:
yes[1].shape

(377,)

In [29]:
FD1x = FM1x/time
FD1y = FM1y/time
FD1z = FM1z/time
FD2xy = FM2xy/time
FD2xz = FM2xz/time
FD2yz = FM2yz/time
FD3 = FM3/time

In [30]:
def plot_MLOG(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
    #I will need to calculate standard deviations eventually
#     #Now to calculate the standard dev at each point:
#     for num in range (1, total1):
#         SDunit = (MSD[num] - MMSD)**2
#         SD = SD + SDunit
#     SD = np.sqrt(SD/total1)
#     SE = SD/np.sqrt(total1)
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
    
#     ax.plot(t, linel[0], linewidth=3, color='blue')
#     ax.plot(t, linel[1], linewidth=3, color='red')
#     ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    plt.yscale('log')
    plt.xscale('log')
    plt.gca().set_xlim([0.1, limit1])
    plt.gca().set_ylim([0.0001, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [31]:
one1, bleh = plot_MLOG(FM2xy, FM1x, FM1y, time, 1, 1, 'MSD (um^2)', logplot, 100.1, 0.1, 1, 1)
plt.show()

In [32]:
def plot_M(M2xy, Mx, My, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linestyle='', linewidth=10, label='2D', marker='o', ms=10, color='blue')
    ax.plot(t, Mx, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='red')
    ax.plot(t, My, linestyle='', linewidth=10, label='1D x', marker='o', ms=10, color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    hfont = {'fontname':'Arial'}
    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'MSD ($\mu$m$^2$)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.05, 0.61), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [33]:
one1, bleh = plot_M(FM2xy, FM1x, FM1y, time, 0, 0, 'MSD (um^2)', Mplot, 50.1, 0.0101, 10, 0.002)
plt.show()

In [34]:
Deff3 = Deff2*np.ones(time.shape[0])

In [35]:
def plot_M1(M2xy, Mx, My, Mth, time, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Plots the MSDs or Deffs from a trajectory dataset.

    n1: particle numbers (typically 0)
    n2: time (typically 1)
    n3: MSDs or Deffs (11 or 15 typically)
    """

    M2 = M2xy
    Mx = Mx
    My = My
    t = time
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t, M2, linewidth=10, label='2D', color='blue')
    ax.plot(t, Mx, linewidth=10, label='1D x', color='red')
    ax.plot(t, My, linewidth=10, label='1D x', color='green')
    ax.plot(t, Mth, '--', linewidth=10, label='theoretical', color='purple')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)

    hfont = {'fontname':'Arial'}
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=115, **hfont)
    ax.set_ylabel(r'D ($\mu$m$^2$/s)', fontsize=115, **hfont)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.10, 0.54), prop={'size': 70})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}e'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return M2, Mx

In [36]:
one1, bleh = plot_M1(FD2xy, FD1x, FD1y, Deff3, time, 0, 1, 'MSD (um^2)', Dplot, 50.1, 0.0021, 10, 0.0004)
plt.show()

In [37]:
SM2xy[34]

masked_array(data = [-- 0.0009610000000003665 0.0007411232000004152 0.0005535360000008032
 0.0027707551999996247 0.0007395855999989457 0.0016636832000009082
 0.0015944912000011072 0.0008918079999998608 0.00015991040000029784
 0.00011224479999978342 0.0005320095999999661 0.0008364544000001047
 0.0015806528000002873 0.0007903264000024696 0.0007165216000016435
 0.0023863552000008875 0.00339809600000224 0.0008272288000007328
 0.0008994960000005122 0.0008687440000004442 0.001094771200000564
 0.0027138640000018525 0.0010163536000008275 0.004097703999997305
 0.0015145360000013812 0.004459040000000291 0.0025216640000005107
 0.0047434959999993 0.002853785599999329 0.004997199999998382
 0.0007995519999989516 0.001459182400000427 0.0009425487999995785
 0.008562894400001783 0.0015145359999993373 0.003779420800000999
 0.0013915279999990647 0.008813523200003564 0.0038778272000005963
 0.00015529759999962503 0.0021710912000015388 0.0032627872000005735
 0.0023448400000010356 0.006288784000002229 0.0387

In [38]:
#Now to calculate arithmetic and geometric means.

arM1x = np.zeros(SM1x[1].shape[0])
arM1y = np.zeros(SM1x[1].shape[0])
arM1z = np.zeros(SM1x[1].shape[0])
arM2xy = np.zeros(SM1x[1].shape[0])
arM2xz = np.zeros(SM1x[1].shape[0])
arM2yz = np.zeros(SM1x[1].shape[0])
arM3 = np.zeros(SM1x[1].shape[0])

arD1x = np.zeros(SM1x[1].shape[0])
arD1y = np.zeros(SM1x[1].shape[0])
arD1z = np.zeros(SM1x[1].shape[0])
arD2xy = np.zeros(SM1x[1].shape[0])
arD2xz = np.zeros(SM1x[1].shape[0])
arD2yz = np.zeros(SM1x[1].shape[0])
arD3 = np.zeros(SM1x[1].shape[0])

st_arM1x = np.zeros(SM1x[1].shape[0])
st_arM1y = np.zeros(SM1x[1].shape[0])
st_arM1z = np.zeros(SM1x[1].shape[0])
st_arM2xy = np.zeros(SM1x[1].shape[0])
st_arM2xz = np.zeros(SM1x[1].shape[0])
st_arM2yz = np.zeros(SM1x[1].shape[0])
st_arM3 = np.zeros(SM1x[1].shape[0])

st_arD1x = np.zeros(SM1x[1].shape[0])
st_arD1y = np.zeros(SM1x[1].shape[0])
st_arD1z = np.zeros(SM1x[1].shape[0])
st_arD2xy = np.zeros(SM1x[1].shape[0])
st_arD2xz = np.zeros(SM1x[1].shape[0])
st_arD2yz = np.zeros(SM1x[1].shape[0])
st_arD3 = np.zeros(SM1x[1].shape[0])

# for num in range(1, tots +1):
#     arM1x = arM1x + SM1x[num]
#     arM1y = arM1y + SM1y[num]
#     arM1z = arM1z + SM1z[num]
#     arM2xy = arM2xy + SM2xy[num]
#     arM2xz = arM2xz + SM2xz[num]
#     arM2yz = arM2yz + SM2yz[num]
#     arM3 = arM3 + SM3[num]
    
#     arD1x = arD1x + SD1x[num]
#     arD1y = arD1y + SD1y[num]
#     arD1z = arD1z + SD1z[num]
#     arD2xy = arD2xy + SD2xy[num]
#     arD2xz = arD2xz + SD2xz[num]
#     arD2yz = arD2yz + SD2yz[num]
#     arD3 = arD3 + SD3[num]
    
#     arM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
#     gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
#     gM1z[num2+1] = ma.masked_invalid(gM1z[num2+1])
#     gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])
#     gM2xz[num2+1] = ma.masked_invalid(gM2xz[num2+1])
#     gM2yz[num2+1] = ma.masked_invalid(gM2yz[num2+1])
#     gM3[num2+1] = ma.masked_invalid(gM3[num2+1])
        
#     gD1x[num2+1] = ma.masked_invalid(SD1x[num2+1])
#     gD1y[num2+1] = ma.masked_invalid(SD1y[num2+1])
#     gD1z[num2+1] = ma.masked_invalid(SD1z[num2+1])
#     gD2xy[num2+1] = ma.masked_invalid(SD2xy[num2+1])
#     gD2xz[num2+1] = ma.masked_invalid(SD2xz[num2+1])
#     gD2yz[num2+1] = ma.masked_invalid(SD2yz[num2+1])
#     gD3[num2+1] = ma.masked_invalid(SD3[num2+1])
    
# arM1x = arM1x/tots
# arM1y = arM1y/tots
# arM1z = arM1z/tots
# arM2xy = arM2xy/tots
# arM2xz = arM2xz/tots
# arM2yz = arM2yz/tots
# arM3 = arM3/tots

# arD1x = arD1x/tots
# arD1y = arD1y/tots
# arD1z = arD1z/tots
# arD2xy = arD2xy/tots
# arD2xz = arD2xz/tots
# arD2yz = arD2yz/tots
# arD3 = arD3/tots

In [39]:
#Two quick fixes to update new nomenclature from old code.
tots = total1 - 1
time2 = time

gM1x = dict()
gM1y = dict()
gM1z = dict()
gM2xy = dict()
gM2xz = dict()
gM2yz = dict()
gM3 = dict()

log_gM1x = dict()
log_gM1y = dict()
log_gM1z = dict()
log_gM2xy = dict()
log_gM2xz = dict()
log_gM2yz = dict()
log_gM3 = dict()

# gD1x = dict()
# gD1y = dict()
# gD1z = dict()
# gD2xy = dict()
# gD2xz = dict()
# gD2yz = dict()
# gD3 = dict()

geoM1x = np.zeros(SM1x[1].shape[0])
geoM1y = np.zeros(SM1x[1].shape[0])
geoM1z = np.zeros(SM1x[1].shape[0])
geoM2xy = np.zeros(SM1x[1].shape[0])
geoM2xz = np.zeros(SM1x[1].shape[0])
geoM2yz = np.zeros(SM1x[1].shape[0])
geoM3 = np.zeros(SM1x[1].shape[0])

st_geoM1x = np.zeros(SM1x[1].shape[0])
st_geoM1y = np.zeros(SM1x[1].shape[0])
st_geoM1z = np.zeros(SM1x[1].shape[0])
st_geoM2xy = np.zeros(SM1x[1].shape[0])
st_geoM2xz = np.zeros(SM1x[1].shape[0])
st_geoM2yz = np.zeros(SM1x[1].shape[0])
st_geoM3 = np.zeros(SM1x[1].shape[0])

geoD1x = np.zeros(SM1x[1].shape[0])
geoD1y = np.zeros(SM1x[1].shape[0])
geoD1z = np.zeros(SM1x[1].shape[0])
geoD2xy = np.zeros(SM1x[1].shape[0])
geoD2xz = np.zeros(SM1x[1].shape[0])
geoD2yz = np.zeros(SM1x[1].shape[0])
geoD3 = np.zeros(SM1x[1].shape[0])

st_geoD1x = np.zeros(SM1x[1].shape[0])
st_geoD1y = np.zeros(SM1x[1].shape[0])
st_geoD1z = np.zeros(SM1x[1].shape[0])
st_geoD2xy = np.zeros(SM1x[1].shape[0])
st_geoD2xz = np.zeros(SM1x[1].shape[0])
st_geoD2yz = np.zeros(SM1x[1].shape[0])
st_geoD3 = np.zeros(SM1x[1].shape[0])

for num2 in range(0, SM1x[1].shape[0]):
    gM1x[num2+1] = np.zeros(tots)
    gM1y[num2+1] = np.zeros(tots)
    gM1z[num2+1] = np.zeros(tots)
    gM2xy[num2+1] = np.zeros(tots)
    gM2xz[num2+1] = np.zeros(tots)
    gM2yz[num2+1] = np.zeros(tots)
    gM3[num2+1] = np.zeros(tots)
    
#     gD1x[num2+1] = np.zeros(tots)
#     gD1y[num2+1] = np.zeros(tots)
#     gD1z[num2+1] = np.zeros(tots)
#     gD2xy[num2+1] = np.zeros(tots)
#     gD2xz[num2+1] = np.zeros(tots)
#     gD2yz[num2+1] = np.zeros(tots)
#     gD3[num2+1] = np.zeros(tots)
    
    for num in range(1, tots + 1):
        gM1x[num2+1][num-1] = SM1x[num][num2]
        gM1y[num2+1][num-1] = SM1y[num][num2]
        gM1z[num2+1][num-1] = SM1z[num][num2]
        gM2xy[num2+1][num-1] = SM2xy[num][num2]
        gM2xz[num2+1][num-1] = SM2xz[num][num2]
        gM2yz[num2+1][num-1] = SM2yz[num][num2]
        gM3[num2+1][num-1] = SM3[num][num2]
        
#         gD1x[num2+1][num-1] = SD1x[num][num2]
#         gD1y[num2+1][num-1] = SD1y[num][num2]
#         gD1z[num2+1][num-1] = SD1z[num][num2]
#         gD2xy[num2+1][num-1] = SD2xy[num][num2]
#         gD2xz[num2+1][num-1] = SD2xz[num][num2]
#         gD2yz[num2+1][num-1] = SD2yz[num][num2]
#         gD3[num2+1][num-1] = SD3[num][num2]
        
    gM1x[num2+1] = ma.masked_invalid(gM1x[num2+1])
    gM1y[num2+1] = ma.masked_invalid(gM1y[num2+1])
    gM1z[num2+1] = ma.masked_invalid(gM1z[num2+1])
    gM2xy[num2+1] = ma.masked_invalid(gM2xy[num2+1])
    gM2xz[num2+1] = ma.masked_invalid(gM2xz[num2+1])
    gM2yz[num2+1] = ma.masked_invalid(gM2yz[num2+1])
    gM3[num2+1] = ma.masked_invalid(gM3[num2+1])

    #I need to include this, because geometric means can't be calculated on values of 0.
    gM1x[num2+1] = ma.masked_equal(gM1x[num2+1], 0)
    gM1y[num2+1] = ma.masked_equal(gM1y[num2+1], 0)
    gM1z[num2+1] = ma.masked_equal(gM1z[num2+1], 0)
    gM2xy[num2+1] = ma.masked_equal(gM2xy[num2+1], 0)
    gM2xz[num2+1] = ma.masked_equal(gM2xz[num2+1], 0)
    gM2yz[num2+1] = ma.masked_equal(gM2yz[num2+1], 0)
    gM3[num2+1] = ma.masked_equal(gM3[num2+1], 0)

    log_gM1x[num2+1] = np.log(gM1x[num2+1])
    log_gM1y[num2+1] = np.log(gM1y[num2+1])
    log_gM1z[num2+1] = np.log(gM1z[num2+1])
    log_gM2xy[num2+1] = np.log(gM2xy[num2+1])
    log_gM2xz[num2+1] = np.log(gM2xz[num2+1])
    log_gM2yz[num2+1] = np.log(gM2yz[num2+1])
    log_gM3[num2+1] = np.log(gM3[num2+1])
        
#     gD1x[num2+1] = ma.masked_invalid(SD1x[num2+1])
#     gD1y[num2+1] = ma.masked_invalid(SD1y[num2+1])
#     gD1z[num2+1] = ma.masked_invalid(SD1z[num2+1])
#     gD2xy[num2+1] = ma.masked_invalid(SD2xy[num2+1])
#     gD2xz[num2+1] = ma.masked_invalid(SD2xz[num2+1])
#     gD2yz[num2+1] = ma.masked_invalid(SD2yz[num2+1])
#     gD3[num2+1] = ma.masked_invalid(SD3[num2+1])
        
    geoM1x[num2] = stat.gmean(gM1x[num2+1])
    geoM1y[num2] = stat.gmean(gM1y[num2+1])
    geoM1z[num2] = stat.gmean(gM1z[num2+1])
    geoM2xy[num2] = stat.gmean(gM2xy[num2+1])
    geoM2xz[num2] = stat.gmean(gM2xz[num2+1])
    geoM2yz[num2] = stat.gmean(gM2yz[num2+1])
    geoM3[num2] = stat.gmean(gM3[num2+1])
    
    #(Wiki method) The geometric standard deviation is a function of the mean of the logarithms.
#     st_geoM1x[num2] = np.exp(np.sqrt(np.mean((log_gM1x[num2+1] - np.log(geoM1x[num2]))*(log_gM1x[num2+1] - np.log(geoM1x[num2])))))
#     st_geoM1y[num2] = np.exp(np.sqrt(np.mean((log_gM1y[num2+1] - np.log(geoM1y[num2]))*(log_gM1y[num2+1] - np.log(geoM1y[num2])))))
#     st_geoM1z[num2] = np.exp(np.sqrt(np.mean((log_gM1z[num2+1] - np.log(geoM1z[num2]))*(log_gM1z[num2+1] - np.log(geoM1z[num2])))))
#     st_geoM2xy[num2] = np.exp(np.sqrt(np.mean((log_gM2xy[num2+1] - np.log(geoM2xy[num2]))*(log_gM2xy[num2+1] - np.log(geoM2xy[num2])))))
#     st_geoM2xz[num2] = np.exp(np.sqrt(np.mean((log_gM2xz[num2+1] - np.log(geoM2xz[num2]))*(log_gM2xz[num2+1] - np.log(geoM2xz[num2])))))
#     st_geoM2yz[num2] = np.exp(np.sqrt(np.mean((log_gM2yz[num2+1] - np.log(geoM2yz[num2]))*(log_gM2yz[num2+1] - np.log(geoM2yz[num2])))))
#     st_geoM3[num2] = np.exp(np.sqrt(np.mean((log_gM3[num2+1] - np.log(geoM3[num2]))*(log_gM3[num2+1] - np.log(geoM3[num2])))))

    #Nance method for standard error or geometric mean.
    st_geoM1x[num2] = np.abs(geoM1x[num2]-np.exp(np.mean(np.log(gM1x[num2+1]))-np.std(np.log(gM1x[num2+1]))/np.sqrt(gM1x[num2+1].shape[0])))
    st_geoM1y[num2] = np.abs(geoM1y[num2]-np.exp(np.mean(np.log(gM1y[num2+1]))-np.std(np.log(gM1y[num2+1]))/np.sqrt(gM1y[num2+1].shape[0])))
    st_geoM1z[num2] = np.abs(geoM1z[num2]-np.exp(np.mean(np.log(gM1z[num2+1]))-np.std(np.log(gM1z[num2+1]))/np.sqrt(gM1z[num2+1].shape[0])))
    st_geoM2xy[num2] = np.abs(geoM2xy[num2]-np.exp(np.mean(np.log(gM2xy[num2+1]))-np.std(np.log(gM2xy[num2+1]))/np.sqrt(gM2xy[num2+1].shape[0])))
    st_geoM2xz[num2] = np.abs(geoM2xz[num2]-np.exp(np.mean(np.log(gM2xz[num2+1]))-np.std(np.log(gM2xz[num2+1]))/np.sqrt(gM2xz[num2+1].shape[0])))
    st_geoM2yz[num2] = np.abs(geoM2yz[num2]-np.exp(np.mean(np.log(gM2yz[num2+1]))-np.std(np.log(gM2yz[num2+1]))/np.sqrt(gM2yz[num2+1].shape[0])))
    st_geoM3[num2] = np.abs(geoM3[num2]-np.exp(np.mean(np.log(gM3[num2+1]))-np.std(np.log(gM3[num2+1]))/np.sqrt(gM3[num2+1].shape[0])))

#     geoD1x[num2] = stat.gmean(gD1x[num2+1])
#     geoD1y[num2] = stat.gmean(gD1y[num2+1])
#     geoD1z[num2] = stat.gmean(gD1z[num2+1])
#     geoD2xy[num2] = stat.gmean(gD2xy[num2+1])
#     geoD2xz[num2] = stat.gmean(gD2xz[num2+1])
#     geoD2yz[num2] = stat.gmean(gD2yz[num2+1])
#     geoD3[num2] = stat.gmean(gD3[num2+1])

    geoD1x[num2] = geoM1x[num2]/time2[num2]
    geoD1y[num2] = geoM1y[num2]/time2[num2]
    geoD1z[num2] = geoM1z[num2]/time2[num2]
    geoD2xy[num2] = geoM2xy[num2]/time2[num2]
    geoD2xz[num2] = geoM2xz[num2]/time2[num2]
    geoD2yz[num2] = geoM2yz[num2]/time2[num2]
    geoD3[num2] = geoM3[num2]/time2[num2]

    arM1x[num2] = np.mean(gM1x[num2+1])
    arM1y[num2] = np.mean(gM1y[num2+1])
    arM1z[num2] = np.mean(gM1z[num2+1])
    arM2xy[num2] = np.mean(gM2xy[num2+1])
    arM2xz[num2] = np.mean(gM2xz[num2+1])
    arM2yz[num2] = np.mean(gM2yz[num2+1])
    arM3[num2] = np.mean(gM3[num2+1])
    
    st_arM1x[num2] = np.std(gM1x[num2+1])
    st_arM1y[num2] = np.std(gM1y[num2+1])
    st_arM1z[num2] = np.std(gM1z[num2+1])
    st_arM2xy[num2] = np.std(gM2xy[num2+1])
    st_arM2xz[num2] = np.std(gM2xz[num2+1])
    st_arM2yz[num2] = np.std(gM2yz[num2+1])
    st_arM3[num2] = np.std(gM3[num2+1])


#     arD1x[num2] = np.mean(gD1x[num2+1])
#     arD1y[num2] = np.mean(gD1y[num2+1])
#     arD1z[num2] = np.mean(gD1z[num2+1])
#     arD2xy[num2] = np.mean(gD2xy[num2+1])
#     arD2xz[num2] = np.mean(gD2xz[num2+1])
#     arD2yz[num2] = np.mean(gD2yz[num2+1])
#     arD3[num2] = np.mean(gD3[num2+1])

    arD1x[num2] = arM1x[num2]/time2[num2]
    arD1y[num2] = arM1y[num2]/time2[num2]
    arD1z[num2] = arM1z[num2]/time2[num2]
    arD2xy[num2] = arM2xy[num2]/time2[num2]
    arD2xz[num2] = arM2xz[num2]/time2[num2]
    arD2yz[num2] = arM2yz[num2]/time2[num2]
    arD3[num2] = arM3[num2]/time2[num2]

    st_arD1x[num2] = st_arM1x[num2]/time2[num2]
    st_arD1y[num2] = st_arM1y[num2]/time2[num2]
    st_arD1z[num2] = st_arM1z[num2]/time2[num2]
    st_arD2xy[num2] = st_arM2xy[num2]/time2[num2]
    st_arD2xz[num2] = st_arM2xz[num2]/time2[num2]
    st_arD2yz[num2] = st_arM2yz[num2]/time2[num2]
    st_arD3[num2] = st_arM3[num2]/time2[num2]

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [40]:
st_arM1x

array([        nan,  0.04384813,  0.04296759,  0.48372214,  0.49231066,
        0.52322695,  0.73562886,  0.78806817,  0.78284984,  0.75650666,
        0.62553545,  0.72679271,  0.68281826,  0.68434636,  0.65611317,
        0.64617514,  0.69542703,  0.64783403,  0.57811291,  0.57958136,
        0.57861137,  0.58117285,  0.04710352,  0.0703788 ,  0.55069667,
        0.60183129,  0.55841421,  0.59614631,  0.59405415,  0.58028679,
        0.61302284,  0.58169943,  0.06003739,  0.08904096,  0.06164975,
        0.06012368,  0.06175655,  0.07421179,  0.07853281,  0.06246955,
        0.0760194 ,  0.0616067 ,  0.08817082,  0.07414516,  0.07380696,
        0.08411842])

In [41]:
#np.log(gM1x[1]) - np.log(geoM1x[0])
#np.exp(np.sqrt(np.log(gM1x[2])*np.log(gM1x[2])))
#st_geoM1x
#np.exp(np.sqrt(np.mean((log_gM1x[4+1] - np.log(geoM1x[4]))*(log_gM1x[4+1] - np.log(geoM1x[4])))))

In [42]:
#SM1x[1].shape
#time2
#np.set_printoptions(precision=4, suppress=True)
# np.where(gM2xy[4]>200)
# gM2xy[20][218]

In [43]:
#np.exp((np.mean(log_gM1x[4+1] - np.log(geoM1x[4])))*(np.mean(log_gM1x[4+1] - np.log(geoM1x[4]))))

In [88]:
def plot_Mcomp(Mgeo, Mar, MFBF, t1, t2, dec1, dec2, datatype, filename, limit1, limit2, tick1, tick2):
    """
    Mgeo = Geometric average dataset
    Mar = Arithmetic average dataset
    MFBF = Frame-by-Frame dataset
    
    t1 = time array for Mgeo and Mar
    t2 time array for MFDF
    """
    
#     #Linear algebra to find Deff:
#     t = time[1][:]
#     w = dict()
#     line = dict()
#     A = np.ones((np.shape(t)[0], 2))
#     A[:, 0] = t
#     w[0] = np.linalg.lstsq(A, MMSD[:, 0])[0]
#     w[1] = np.linalg.lstsq(A, MMSD[:, 1])[0]
#     w[2] = np.linalg.lstsq(A, MMSD[:, 2])[0]
#     line[0] = w[0][0]*t + w[0][1]
#     line[1] = w[1][0]*t + w[1][1]
#     line[2] = w[2][0]*t + w[2][1]
    
#     #Linear algebra for fit on log plot:
#     wl = dict()
#     linel = dict()
#     lt = np.log(t)
#     lA = np.ones((np.shape(t)[0], 2))
#     lA[:, 0] = lt
#     lM = np.log(MMSD)
#     wl[0] = np.linalg.lstsq(lA, lM[:, 0])[0]
#     wl[1] = np.linalg.lstsq(lA, lM[:, 1])[0]
#     wl[2] = np.linalg.lstsq(lA, lM[:, 2])[0]
#     linel[0] = np.exp(wl[0][0]*lt + wl[0][1])
#     linel[1] = np.exp(wl[1][0]*lt + wl[1][1])
#     linel[2] = np.exp(wl[2][0]*lt + wl[2][1])

    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    ax.plot(t1, Mgeo, linewidth=10, label='Geometric', color='blue')
    ax.plot(t1, Mar, linewidth=10, label='Arithmetic', color='red')
    ax.plot(t2, MFBF, linewidth=10, label='Frame-by-Frame', color='green')
    
#     ax.errorbar(t, MMSD[:, 0], yerr=SE[:, 0], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='blue')
#     ax.errorbar(t, MMSD[:, 1], yerr=SE[:, 1], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='red')
#     ax.errorbar(t, MMSD[:, 2], yerr=SE[:, 2], fmt='', linestyle='', capsize=7, capthick=2, elinewidth=2, color='green')
    
#     ax.plot(t, line[0], linewidth=3, color='blue')
#     ax.plot(t, line[1], linewidth=3, color='red')
#     ax.plot(t, line[2], linewidth=3, color='green')
    
    #ax.plot(t, linel[0], linewidth=3, color='blue')
    #ax.plot(t, linel[1], linewidth=3, color='red')
    #ax.plot(t, linel[2], linewidth=3, color='green')
    
    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(0, limit1, tick1)
    ymajor_ticks = np.arange(0, limit2, tick2)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('Time (s)', fontsize=95)
    ax.set_ylabel(datatype, fontsize=95)
    ax.tick_params(direction='out', pad=16)
    ax.legend(loc=(0.60, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec1)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec2)))

    
    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([0, limit1])
    plt.gca().set_ylim([0, limit2])

    # Save your figure
    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return Mar, Mgeo

In [89]:
one1, bleh = plot_Mcomp(geoM2xy, arM2xy, FM2xy, time2, time, 0, 2, 'MSD (um^2)', Cplot, 50.1, 0.021, 10, 0.004)
plt.show()

In [46]:
np.set_printoptions(precision=5, suppress=True)
np.round(arM1x, 5)

array([     nan,  0.01964,  0.01961,  0.04202,  0.04369,  0.04388,
        0.06905,  0.08258,  0.09612,  0.086  ,  0.06857,  0.08179,
        0.06854,  0.06791,  0.0665 ,  0.0648 ,  0.07058,  0.06942,
        0.04917,  0.05282,  0.05038,  0.05021,  0.02125,  0.02587,
        0.05836,  0.05938,  0.05462,  0.05748,  0.0553 ,  0.05431,
        0.06312,  0.05889,  0.02392,  0.02908,  0.02463,  0.0253 ,
        0.02549,  0.03285,  0.03049,  0.02999,  0.03103,  0.02662,
        0.03318,  0.02873,  0.02749,  0.03223])

In [47]:
np.round(geoM1x, 5)

array([     nan,  0.0032 ,  0.0022 ,  0.00269,  0.00252,  0.00276,
        0.00269,  0.00267,  0.00213,  0.00232,  0.00261,  0.00257,
        0.00263,  0.00273,  0.00298,  0.00235,  0.00264,  0.00322,
        0.00252,  0.00287,  0.00262,  0.00302,  0.00308,  0.00341,
        0.00368,  0.00326,  0.00373,  0.00347,  0.0031 ,  0.0032 ,
        0.00442,  0.00403,  0.00293,  0.00297,  0.0038 ,  0.00377,
        0.003  ,  0.00364,  0.0043 ,  0.00463,  0.00445,  0.00461,
        0.00387,  0.00439,  0.0036 ,  0.00465])

In [48]:
np.round(FM1x, 5)

array([     nan,  0.00241,  0.0014 ,  0.0019 ,  0.00204,  0.00215,
        0.002  ,  0.00206,  0.00167,  0.00193,  0.00229,  0.00214,
        0.00218,  0.00233,  0.00245,  0.0023 ,  0.00263,  0.00314,
        0.00246,  0.00278,  0.00274,  0.0028 ,  0.00327,  0.00324,
        0.00379,  0.00346,  0.00354,  0.00351,  0.00324,  0.00342,
        0.00436,  0.00409,  0.00317,  0.00321,  0.00379,  0.00412,
        0.003  ,  0.00378,  0.00419,  0.00465,  0.00439,  0.00488,
        0.00399,  0.00449,  0.0037 ,  0.00488])

In [49]:
#Check for faulty particle trajectories here.

at_fault = np.where(gM2xy[3]>0.4)[0] + 1
# gM1x[2][34]
# gM1x[2][218]
at_fault
#M2xy[at_fault[1]+1]

array([], dtype=int64)

In [50]:
def plot_histogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """
    total1 = total
    MSD = M

    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]

    hist = [x for x in hist if str(x) != 'nan']
    
    plot, bins = np.histogram(hist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
#     plt.xlabel('D (um2/s)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [51]:
def find_nearest(array, value):
    idx = (np.abs(array-value)).argmin()
    return array[idx], idx

time_a, tindex = find_nearest(time2, 1)
geoDat1 = geoD2xy[tindex]
arDat1 = arD2xy[tindex]

time_a1, tindex1 = find_nearest(time, 1)
FDat1 = FD2xy[tindex1]

print('geometric average at 1s:', geoDat1, 'um2/s with', tots, 'particles')
print('arithmetic average at 1s:', arDat1, 'um2/s with', tots, 'particles')
print('frame-by-frame average at 1s:', FDat1, 'um2/s with', fifties, 'particles')

geometric average at 1s: 0.00937249339625 um2/s with 542 particles
arithmetic average at 1s: 0.038038398348 um2/s with 542 particles
frame-by-frame average at 1s: 0.00725133184479 um2/s with 542 particles


In [52]:
a, hist, total= plot_histogram(SD2xy, time2, tots, 'Deff (um^2/s)', Hplot, 1)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [53]:
def plot_loghistogram(M, time, total, datatype, filename, tn):
    """
    Plots the MSDs or Deffs from a trajectory dataset.
    """

    total1 = total
    MSD = M
    
    def find_nearest(array, value):
        idx = (np.abs(array-value)).argmin()
        return array[idx], idx

    td, idx = find_nearest(time, tn)
    
    hist = np.zeros(total1)
    for num in range(1, total1):
        hist[num] = MSD[num][idx]
    
    z1, z2 = find_nearest(hist[1:], 0)
    hist = np.delete(hist[1:], z2)
    hist = [x for x in hist if str(x) != 'nan']
    loghist = np.log(hist)

    plot, bins = np.histogram(loghist, bins=20)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:])/2
    plt.bar(center, plot, align='center', width=width)
    plt.xlabel('log(D)', fontsize=20)

    plt.savefig('{}.png'.format(filename), bbox_inches='tight')
    return hist, MSD, total1

In [54]:
hist, MSD, total = plot_loghistogram(SD2xy, time2, tots, 'Deff (um^2/s)', Hlogplot, 1)
plt.show()

C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [55]:
np.savetxt('geoD2xy_{}.csv'.format(name), geoD2xy, delimiter=',')
np.savetxt('arD2xy_{}.csv'.format(name), arD2xy, delimiter=',')
np.savetxt('FD2xy_{}.csv'.format(name), FD2xy, delimiter=',')

np.savetxt('geoM2xy_{}.csv'.format(name), geoM2xy, delimiter=',')
np.savetxt('arM2xy_{}.csv'.format(name), arM2xy, delimiter=',')
np.savetxt('FM2xy_{}.csv'.format(name), FM2xy, delimiter=',')

In [56]:
ex1 = trajectory[1]
ex2 = trajectory[1]
ex3 = trajectory[1]

In [57]:
gM1x[2]

masked_array(data = [0.0001537600000003406 0.004319118399999854 0.001293121600000207 --
 0.001992729600001369 0.00911643040000174 0.001477633600000651
 3.844000000008515e-05 7.534239999996953e-05 1.5375999999893086e-06
 0.012732865599988435 0.004651240000002549 0.0024601599999998103
 0.1393080976000073 0.10797795999997767 0.0007441983999994775
 0.00012454560000040276 1.3838399999903778e-05 0.0038439999999944174
 0.03144238240000312 3.843999999990893e-05 0.0038439999999944174
 0.0006150399999985429 0.0015745024000025855 0.0018835599999992383
 0.0002214144000001521 0.0408524944000003 0.00297679359999791
 0.0024601599999998103 0.0004981823999990735 0.015375999999998816 --
 0.003844000000001466 -- 0.017276473599999415 -- 0.0007441983999979268
 0.25345183359998336 0.0005550735999988189 0.00034595999999970903
 0.001293121599999185 0.001039417599999645 0.0006780815999997258
 0.015070017599992908 0.005172486400004916 0.014767110399994027
 0.06780816000001698 0.1584358415999957 0.00167444640000

In [58]:
Tplot = '{}_Tplot'.format(name)

In [59]:
maxx = 290
minx = 200
maxy = 290
miny = 200

def plot_traj(xmax, xmin, ymax, ymin, xts, yts):
    # Creates figure
    fig = plt.figure(figsize=(24, 18), dpi=80)
    ax = fig.add_subplot(111)
    # ax.set_title('Particle Trajectories', x=0.5, y=1.15)

    for num in range(1, total1):
        ax.plot(xts[num], yts[num], linewidth=10, label='Particle {}'.format(num))


    # A few adjustments to prettify the graph
    for item in ([ax.xaxis.label, ax.yaxis.label] +
                 ax.get_xticklabels() + ax.get_yticklabels()):
        item.set_fontsize(70)

    xmajor_ticks = np.arange(xmin, xmax + 0.001, 20)
    ymajor_ticks = np.arange(ymin, ymax + 0.001, 20)

    ax.set_xticks(xmajor_ticks)
    ax.set_yticks(ymajor_ticks)
    ax.title.set_fontsize(70)
    ax.set_xlabel('x (um)', fontsize=95)
    ax.set_ylabel('y (um)', fontsize=95)
    #ax.tick_params(direction='out', pad=16)
    #ax.legend(loc=(0.60, 0.76), prop={'size': 40})
    plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(0)))


    #plt.yscale('log')
    #plt.xscale('log')
    plt.gca().set_xlim([xmin, xmax])
    plt.gca().set_ylim([ymin, ymax])

    # Save your figure
    plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')

plot_traj(maxx, minx, maxy, miny, xs, ys)
plt.show()

In [75]:
unit = 4

maxx = 0.1*unit
minx = -0.1*unit
maxy = 0.1*unit
miny = -0.1*unit
ticks = 0.04*unit
dec = 4

NPs = np.array([1, 10, 20, 30, 40, 50])
#NPs = filte
#NPs = np.array([30])
xc = dict()
yc = dict()

# Creates figure
fig1 = plt.figure(figsize=(24, 18), dpi=80)
ax1 = fig1.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)

for num in range(1, NPs.shape[0]+1):
    lowx = min(xs[NPs[num-1]])
    highx = max(xs[NPs[num-1]])
    lowy = min(ys[NPs[num-1]])
    highy = max(ys[NPs[num-1]])
    
    xc[NPs[num-1]] = np.array([xs[NPs[num-1]] - ((highx+lowx)/2)])
    yc[NPs[num-1]] = np.array([ys[NPs[num-1]] - ((highy+lowy)/2)])
    
    ax1.plot(xc[NPs[num-1]][0, :], yc[NPs[num-1]][0, :], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# A few adjustments to prettify the graph
for item in ([ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(minx, maxx + 0.00001, ticks)
ymajor_ticks = np.arange(miny, maxy + 0.00001, ticks)

ax1.set_xticks(xmajor_ticks)
ax1.set_yticks(ymajor_ticks)
ax1.title.set_fontsize(70)
ax1.set_xlabel('x (um)', fontsize=95)
ax1.set_ylabel('y (um)', fontsize=95)
ax1.tick_params(direction='out', pad=16)
plt.xticks(rotation=-30)
#ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(dec)))


#plt.yscale('log')
#plt.xscale('log')
plt.gca().set_xlim([minx, maxx])
plt.gca().set_ylim([miny, maxy])

# Save your figure
plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
plt.show()

In [70]:
#Next, I will make a code to calculate MSD the right way-- as a function of lag time instead of just time.
#This block can calculate MSD(tau) for a single particle chosen by the user.

yourNP = 20
m1x = np.zeros(frames.shape[0])
m1xa = dict()

for num1 in range(1, frames.shape[0]):

    tau = num1
    m1xa[num1] = np.zeros(frames.shape[0] - tau)
    for num in range(0, frames.shape[0] - tau):
        m1xa[num1][num] = (xs[yourNP][num + tau] - xs[yourNP][num])**2 + (ys[yourNP][num + tau] - ys[yourNP][num])**2
    
    m1xa[num1] = ma.masked_invalid(m1xa[num1])
    m1x[num1] = np.mean(m1xa[num1])

m1x = ma.masked_invalid(m1x)

In [83]:

#For normal scaling
# maxx = 45
# minx = 0
# maxy = 0.1
# miny = 0
# xticks = 10
# yticks = 0.02
# xdec = 0
# ydec = 2

#For log scaling
maxx = 100
minx = 0.1
maxy = 1
miny = 0.001
xticks = 10
yticks = 0.02
xdec = 0
ydec = 2

NPs = np.array([1, 10, 20, 30, 40, 50])
#NPs = filte
#NPs = np.array([30])
xc = dict()
yc = dict()

# Creates figure
fig1 = plt.figure(figsize=(24, 18), dpi=80)
ax1 = fig1.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)
    
ax1.plot(time[1:], m1x[1:], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# A few adjustments to prettify the graph
for item in ([ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(minx, maxx + 0.00001, xticks)
ymajor_ticks = np.arange(miny, maxy + 0.00001, yticks)

ax1.set_xticks(xmajor_ticks)
ax1.set_yticks(ymajor_ticks)
ax1.title.set_fontsize(70)
ax1.set_xlabel('x (um)', fontsize=95)
ax1.set_ylabel('y (um)', fontsize=95)
ax1.tick_params(direction='out', pad=16)
plt.xticks(rotation=-30)
#ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(xdec)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(ydec)))


plt.yscale('log')
plt.xscale('log')
plt.gca().set_xlim([minx, maxx])
plt.gca().set_ylim([miny, maxy])

# Save your figure
plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
plt.show()

In [65]:
am1x = dict()

m1xa = dict()

a1 = np.zeros((total1, frames.shape[0]))

for num2 in range(1, total1):

    am1x[num2] = np.zeros(frames.shape[0])

    for num1 in range(1, frames.shape[0]):

        tau = num1
        m1xa[num1] = np.zeros(frames.shape[0] - tau)
        for num in range(0, frames.shape[0] - tau):
            m1xa[num1][num] = (xs[num2][num + tau] - xs[num2][num])**2 + (ys[num2][num + tau] - ys[num2][num])**2
            
        m1xa[num1] = ma.masked_invalid(m1xa[num1])
        am1x[num2][num1] = np.mean(m1xa[num1])

    am1x[num2] = ma.masked_invalid(am1x[num2])
    a1[num2-1, :] = am1x[num2]

a1 = ma.masked_invalid(a1)
tot_m1x = np.mean(a1, axis=0)



C:\Users\koolk\Miniconda3\lib\site-packages\numpy\ma\core.py:4085: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


In [68]:
a1.shape

(543, 46)

In [69]:
filte = np.where(a1[:, 5]>0.5)[0]+1
filte

array([], dtype=int64)

In [94]:
#For normal scaling
maxx = 45
minx = 0
maxy = 0.06
miny = 0.03
xticks = 10
yticks = 0.01
xdec = 0
ydec = 2

#For log scaling
# maxx = 100
# minx = 1
# maxy = 0.1
# miny = 0.01
# xticks = 10
# yticks = 0.02
# xdec = 0
# ydec = 2

NPs = np.array([1, 10, 20, 30, 40, 50])
#NPs = filte
#NPs = np.array([30])
xc = dict()
yc = dict()

# Creates figure
fig1 = plt.figure(figsize=(24, 18), dpi=80)
ax1 = fig1.add_subplot(111)
# ax.set_title('Particle Trajectories', x=0.5, y=1.15)
    
ax1.plot(time[1:], tot_m1x[1:], linewidth=10, label='Particle {}'.format(NPs[num-1]))


# A few adjustments to prettify the graph
for item in ([ax1.xaxis.label, ax1.yaxis.label] +
             ax1.get_xticklabels() + ax1.get_yticklabels()):
    item.set_fontsize(70)

xmajor_ticks = np.arange(minx, maxx + 0.00001, xticks)
ymajor_ticks = np.arange(miny, maxy + 0.00001, yticks)

ax1.set_xticks(xmajor_ticks)
ax1.set_yticks(ymajor_ticks)
ax1.title.set_fontsize(70)
ax1.set_xlabel('x (um)', fontsize=95)
ax1.set_ylabel('y (um)', fontsize=95)
ax1.tick_params(direction='out', pad=16)
plt.xticks(rotation=-30)
#ax1.legend(loc=(0.60, 0.46), prop={'size': 40})
plt.gca().xaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(xdec)))
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter('%.{}f'.format(ydec)))


# plt.yscale('log')
# plt.xscale('log')
plt.gca().set_xlim([minx, maxx])
plt.gca().set_ylim([miny, maxy])

# Save your figure
plt.savefig('{}.png'.format(Tplot), bbox_inches='tight')
plt.show()